# Louis de Broglie and the world’s most important PhD thesis

Louis de Broglie began by assigning an extremely small mass to the photon, so that while this implied that the velocity of the light varied with its frequency, the mass was so small that the deviation from c would remain undetectable. This assumption is not really necessary, but we will follow de Broglie in order to see how his thinking developed. In a way that was perhaps fortuitous, this assumption meant that it was just a small step to apply these ideas to
massive particles.

## Code Preamble

In [ ]:
(ns de-broglie
    (:refer-clojure :exclude [+ - * / zero? compare divide numerator denominator infinite? abs ref partial = time])
    (:require [emmy.env :refer :all]
              [scicloj.kindly.v4.kind :as kind]))

In [ ]:
  (defn walk [inner outer form]
    (cond
      (list? form) (outer (apply list (map inner form)))
      (seq? form)  (outer (doall (map inner form)))
      (coll? form) (outer (into (empty form) (map inner form)))
      :else        (outer form)))
  (defn postwalk [f form]
    (walk (partial postwalk f) f form))
  (defn postwalk-replace [smap form]
    (postwalk (fn [x] (if (contains? smap x) (smap x) x)) form))
  (defmacro let-scheme [b & e]
    (concat (list 'let (into [] (apply concat b))) e))
  (defmacro define-1 [h & b]
    (let [body (postwalk-replace {'let 'let-scheme} b)]
      (if (coll? h)
        (if (coll? (first h))
          (list 'defn (ffirst h) (into [] (rest (first h)))
                (concat (list 'fn (into [] (rest h))) body))
          (concat (list 'defn (first h) (into [] (rest h)))
                  body))
        (concat (list 'def h) body))))
  (defmacro define [h & b]
    (if (and (coll? h) (= (first h) 'tex-inspect))
      (list 'do
            (concat ['define-1 (second h)] b)
            h)
      (concat ['define-1 h] b)))
  (defmacro lambda [h b]
    (list 'fn (into [] h) b))
  (def show-expression simplify)
  (def velocities velocity)
  (def coordinates coordinate)
  (def vector-length count)
  (defn time [state] (first state))

In [ ]:
(defn texv [coll]
    (->>
        (map (fn [e] (->TeX e)) coll)
        (interpose " = ")
        (apply str)
        (#(str "$" % "$"))
        (kind/tex)))

In [ ]:
(defn texvv [coll1 coll2]
    (let [ok (map (fn [x y] (zero? (simplify (- x y)))) coll1 coll2)]
    (if (every? true? ok)
        (texv coll1)
        (doall ok))))

In [ ]:
(defn solves [prefix exp2 fun]
    (if (zero? (simplify (fun exp2)))
        (texv (up prefix exp2))
        "no root"))

## Particles accompanied by oscillatory phenomena

A photon of mass m would have a rest energy of $mc^2$.

In [ ]:
(define (E0 m) (* m (expt 'c 2)))

(texv (up 'E_0 (E0 'm)))

This should be associated with some kind of periodic vibration of frequency $\nu_0 = mc^2/h$.

In [ ]:
(define (nu_0 E0) (/ E0 'h))

(texv (up 'nu_0 (nu_0 (E0 'm))))

In a frame in which the photon was moving with velocity v = βc (lab frame),

In [ ]:
(define (v beta) (* beta 'c))
(define (sqr beta) (expt beta 2))
(define (sqrt-beta beta) (expt (- 1 (sqr beta)) (/ 1 2)))

(texv (up 'v (v 'beta)))

the energy and frequency are given by the Lorentz transformation:

In [ ]:
(texv (up 'E (/ (E0 'm) (sqrt-beta 'beta))))

In [ ]:
(define ((nu nu_0) beta) (/ nu_0 (sqrt-beta beta)))

(texv (up 'nu ((nu 'nu_0) 'beta)))

On the other hand, due to time dilation we would expect something vibrating with frequency νo in the rest frame to have a frequency

In [ ]:
(define (nu_1 nu_0 beta) (* nu_0 (sqrt-beta beta)))

(texv (up 'nu_1 (nu_1 'nu_0 'beta)))

as seen by an observer in the lab frame. What is going on here? Most people would have concluded at this point that there was something wrong with the whole idea and started working on something else. But not de Broglie. In addition to the internal vibration, he assumed that the particle was also associated with a wave of frequency $\nu$ and phase velocity $v_p$. Thus the wave would be described by a function

In [ ]:
(define ((wave nu v_p) x t) (sin (* 2 'pi nu (- t (/ x v_p)))))
(texv (up ((wave 'nu 'v_p) 'x 't)))

If the particle (remember we are dealing with photons here) was at x = 0 at t = 0 and the wave and internal vibration were in phase at that time, at a time t later (in the lab frame), the particle would be at x = vt, the wave would be given by

In [ ]:
;; the "wave" at the position of the particle is more a vibration, 
;; it only depends on time t, not on x anymore

(define ((external-vibration nu v_p v) t) 
        ((wave nu v_p) (* v t) t))

(texvv (up (sin (* 2 'pi 'nu 't (- 1 (/ 'v 'v_p) )))) 
       (up ((external-vibration 'nu 'v_p 'v) 't)))

and the internal vibration would be at a phase

In [ ]:
(define ((internal-vibration nu_1) t) (sin (* 2 'pi nu_1 t)))
(texv (up ((internal-vibration 'nu_1) 't)))

The wave and the internal
vibration would always be in phase if

In [ ]:
(define ((phase-difference nu v_p v t) nu_1)
        (- ((internal-vibration nu_1) t)
           ((external-vibration nu v_p v) t)))

(define (nu_1_B nu v_p v) (* nu (- 1 (/ v v_p))))

(solves 'nu_1 
        (nu_1_B 'nu 'v_p 'v) 
        (phase-difference 'nu 'v_p 'v 't))

In [ ]:
(define (equation_nu_1 v beta)
    (up
     (nu_1 'nu_0 beta) 
     (nu_1_B ((nu 'nu_0) beta) 'v_p v)))

(texvv
 (up
  (* 'nu_0 (sqrt-beta 'beta))
  (* (/ 'nu_0 (sqrt-beta 'beta)) (- 1 (/ 'v 'v_p))))
 (equation_nu_1 'v 'beta))

which means that

In [ ]:
(define (transform1 form beta)
        (-> form
            (* (/ -1 'nu_0))
            (* (sqrt-beta beta))
            (+ (up 1 1))
            simplify))

(texv (transform1 (equation_nu_1 'v 'beta) 'beta))

and so

In [ ]:
(define (transform2 form beta)
        (-> form
            (transform1 beta)
            (* 'v_p)
            (/ (sqr beta))
            simplify))

(texv (transform2 (equation_nu_1 (v 'beta) 'beta) 'beta))

which is $> c$. Thus he concluded that the wave could not transport energy. He initially considered it as a fictional wave associated with the motion and called it a “phase wave.”

The group velocity in one dimension: 
$$v_g 
= \frac{\mathrm{d}\omega}{\mathrm{d}k}
= \frac{\mathrm{d}\nu}{\mathrm{d}(1/\lambda)}
= \frac{\mathrm{d}\nu}{\mathrm{d}(\nu/v_p)}
= \frac{\mathrm{d}\nu / \mathrm{d}\beta}{\mathrm{d}(\nu/v_p) / \mathrm{d}\beta}
$$

We have

In [ ]:
(define (v_p beta) (second (transform2 (equation_nu_1 (v beta) beta) beta)))

(define ((inv-lambda m) beta) 
        (/ ((nu (nu_0 (E0 m))) beta) (v_p beta)))

(texvv (up (/ 'nu 'v_p) 
           (/ (* 'm 'beta 'c) (* 'h (sqrt-beta 'beta)))) 
      (up (/ 'nu 'v_p)
          ((inv-lambda 'm) 'beta)))

and we find

In [ ]:
(define v_g 
    (/ (D (nu (nu_0 (E0 'm)))) 
       (D (inv-lambda 'm))))

(texv (up 'v_g (simplify (v_g 'beta))))

which is the particle velocity. Another way of seeing this is to note that if H (p, q) = hν = ħω is the total energy, the particle velocity is given by Hamilton’s equation:

$$\dot{q} 
= \frac{\partial{H}}{\partial{p}}
= \frac{\partial{(\hbar \omega)}}{\partial{(\hbar k)}}
= \frac{\partial{\omega}}{\partial{k}}
= v_g
$$

where we used the de Broglie relation p = ħk